In [ ]:
import numpy as np
from pathlib import Path
import librosa
from collections import defaultdict
import soundfile as sf
import tqdm
import os
from multiprocessing import Pool, cpu_count
import cv2
from matplotlib import pyplot as plt
import IPython
import pandas as pd
import scipy.signal

%matplotlib inline

In [ ]:
INPUT =  "../input/resample_sound_2"
OUTPUT = "../output/train_img_3"
SAMPLE_RATE = 32_000
MU = 256
NUM_WORKERS = cpu_count()

print(NUM_WORKERS)

In [ ]:
def audio_to_spec(audio):
    spec = librosa.power_to_db(
        librosa.feature.melspectrogram(audio, sr=SAMPLE_RATE, fmin=20, fmax=16000, n_mels=128)
    )
    return spec.astype(np.float32)

def audio2vec(path):
    x, _ = sf.read(path)
    x_spex = audio_to_spec(x)
    np.save(f"{OUTPUT}/{path.parent.name}/{path.name}.npz", x_spex)
    
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def audio2pict(path):
    x, _ = sf.read(path)
    x_spex = audio_to_spec(x)
    cv2.imwrite(f"{OUTPUT}/{path.parent.name}/{path.name}.jpg", mono_to_color(x_spex))
    
def audio2quantized(path):
    data, _ = librosa.load(path=path, sr=SAMPLE_RATE, mono=True)
    mu_x = np.sign(data) * np.log(1 + MU * np.abs(data)) / np.log(MU + 1)
    bins = np.linspace(-1, 1, MU)
    quantized = np.digitize(mu_x, bins) - 1
    quantized = quantized.astype(np.uint8)
    np.save(f"{OUTPUT}/{path.parent.name}/{path.name}.npz", quantized)

In [ ]:
import time
from datetime import timedelta as td


def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length, win_length)


def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)


def _db_to_amp(x,):
    return librosa.core.db_to_amplitude(x, ref=1.0)


def plot_spectrogram(signal, title):
    fig, ax = plt.subplots(figsize=(20, 4))
    cax = ax.matshow(
        signal,
        origin="lower",
        aspect="auto",
        cmap=plt.cm.seismic,
        vmin=-1 * np.max(np.abs(signal)),
        vmax=np.max(np.abs(signal)),
    )
    fig.colorbar(cax)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


def plot_statistics_and_filter(
    mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
):
    fig, ax = plt.subplots(ncols=2, figsize=(20, 4))
    plt_mean, = ax[0].plot(mean_freq_noise, label="Mean power of noise")
    plt_std, = ax[0].plot(std_freq_noise, label="Std. power of noise")
    plt_std, = ax[0].plot(noise_thresh, label="Noise threshold (by frequency)")
    ax[0].set_title("Threshold for mask")
    ax[0].legend()
    cax = ax[1].matshow(smoothing_filter, origin="lower")
    fig.colorbar(cax)
    ax[1].set_title("Filter for smoothing Mask")
    plt.show()


def removeNoise(
    audio_clip,
    noise_clip,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False,
):
    """Remove noise from audio based upon a clip containing only noise

    Args:
        audio_clip (array): The first parameter.
        noise_clip (array): The second parameter.
        n_grad_freq (int): how many frequency channels to smooth over with the mask.
        n_grad_time (int): how many time channels to smooth over with the mask.
        n_fft (int): number audio of frames between STFT columns.
        win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
        hop_length (int):number audio of frames between STFT columns.
        n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
        prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)
        visual (bool): Whether to plot the steps of the algorithm

    Returns:
        array: The recovered signal with noise subtracted

    """
    if verbose:
        start = time.time()
    # STFT over noise
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))  # convert to dB
    # Calculate statistics over noise
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh
    if verbose:
        print("STFT on noise:", td(seconds=time.time() - start))
        start = time.time()
    # STFT over signal
    if verbose:
        start = time.time()
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))
    if verbose:
        print("STFT on signal:", td(seconds=time.time() - start))
        start = time.time()
    # Calculate value to mask dB to
    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    if verbose:
        print(noise_thresh, mask_gain_dB)
    # Create a smoothing filter for the mask in time and frequency
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    # calculate the threshold for each frequency/time bin
    db_thresh = np.repeat(
        np.reshape(noise_thresh, [1, len(mean_freq_noise)]),
        np.shape(sig_stft_db)[1],
        axis=0,
    ).T
    # mask if the signal is above the threshold
    sig_mask = sig_stft_db < db_thresh
    if verbose:
        print("Masking:", td(seconds=time.time() - start))
        start = time.time()
    # convolve the mask with a smoothing filter
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease
    if verbose:
        print("Mask convolution:", td(seconds=time.time() - start))
        start = time.time()
    # mask the signal
    sig_stft_db_masked = (
        sig_stft_db * (1 - sig_mask)
        + np.ones(np.shape(mask_gain_dB)) * mask_gain_dB * sig_mask
    )  # mask real
    sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (
        1j * sig_imag_masked
    )
    if verbose:
        print("Mask application:", td(seconds=time.time() - start))
        start = time.time()
    # recover the signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)
    recovered_spec = _amp_to_db(
        np.abs(_stft(recovered_signal, n_fft, hop_length, win_length))
    )
    if verbose:
        print("Signal recovery:", td(seconds=time.time() - start))
    if visual:
        plot_spectrogram(noise_stft_db, title="Noise")
    if visual:
        plot_statistics_and_filter(
            mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
        )
    if visual:
        plot_spectrogram(sig_stft_db, title="Signal")
    if visual:
        plot_spectrogram(sig_mask, title="Mask applied")
    if visual:
        plot_spectrogram(sig_stft_db_masked, title="Masked signal")
    if visual:
        plot_spectrogram(recovered_spec, title="Recovered spectrogram")
    return recovered_signal

def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/20),min_periods=1,center=True).max()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean

def audio2pict_denonise(path):
    x, _ = librosa.load(path=path, sr=SAMPLE_RATE, mono=True)
    mask, env = envelope(x, SAMPLE_RATE, threshold=0.05)
    try:
        x = removeNoise(audio_clip=x, noise_clip=x[np.logical_not(mask)],verbose=False,visual=False)
    except:
        pass
    x_spex = audio_to_spec(x)
    cv2.imwrite(f"{OUTPUT}/{path.parent.name}/{path.name}.jpg", mono_to_color(x_spex))

# Noise Delete

In [ ]:
train_df = pd.read_csv("../input/train.csv")

recs = defaultdict(list)
for directory in Path(INPUT).iterdir():
    if directory.name in [".DS_Store"]:
        continue
    dir_paths = [f for f in directory.iterdir() if f.name not in [".DS_Store", "train_mod.csv"]]
    for dname in tqdm.tqdm_notebook(dir_paths, total=len(dir_paths)):
        file_paths = [f for f in dname.iterdir() if f.name != ".DS_Store"]
        break
    break

In [ ]:
path = file_paths[0]
fname = path.name.split(".")[0]
display(train_df.query(f"filename=='{fname}.mp3'")[["rating", "type", "primary_label", "secondary_labels"]])
x, _ = librosa.load(path=path, sr=SAMPLE_RATE, mono=True)
x_spex = audio_to_spec(x)
plt.plot(x);plt.show()
plt.imshow(mono_to_color(x_spex))
IPython.display.Audio(data=x, rate=SAMPLE_RATE)

In [ ]:
mask, env = envelope(x, SAMPLE_RATE, threshold=0.05)
plt.plot(x[mask])
plt.plot(x[np.logical_not(mask)]);plt.show()

#plt.imshow(mono_to_color(audio_to_spec(x[mask])));plt.show()
#plt.imshow(mono_to_color(audio_to_spec(x[np.logical_not(mask)])));plt.show()

#IPython.display.Audio(data=x[mask], rate=SAMPLE_RATE)
#IPython.display.Audio(data=x[np.logical_not(mask)], rate=SAMPLE_RATE)

In [ ]:
output = removeNoise(audio_clip=x, noise_clip=x[np.logical_not(mask)],verbose=True,visual=True)
#output = removeNoise(audio_clip=x, noise_clip=x[mask],verbose=True,visual=True)

IPython.display.Audio(data=output, rate=SAMPLE_RATE)

In [ ]:
plt.plot(output)

In [ ]:
plt.imshow(x_spex[:, :1000])

In [ ]:
is_call = (pd.Series(mask).rolling(SAMPLE_RATE*5).max() == 1.0).values

In [ ]:
plt.plot(is_call)

In [ ]:
plt.imshow(audio_to_spec(output)[:, :1000])

In [ ]:
#mask, env = envelope(output, SAMPLE_RATE, threshold=0.05)
#plt.plot(output[mask]);plt.show()
#plt.imshow(mono_to_color(audio_to_spec(output[mask])));plt.show()
#IPython.display.Audio(data=output[mask], rate=SAMPLE_RATE)

In [ ]:
recs = defaultdict(list)
for directory in Path(INPUT).iterdir():
    if directory.name in [".DS_Store"]:
        continue
    dir_paths = [f for f in directory.iterdir() if f.name not in [".DS_Store", "train_mod.csv"]]
    for dname in tqdm.tqdm_notebook(dir_paths, total=len(dir_paths)):
        file_paths = [f for f in dname.iterdir() if f.name != ".DS_Store"]
        !mkdir -p "{OUTPUT}/{dname.name}"
        with Pool(NUM_WORKERS // 2) as p:
            #p.map(audio2pict, file_paths)
            p.map(audio2pict_denonise, file_paths)

In [ ]:
recs = defaultdict(list)
for directory in tqdm.tqdm_notebook(Path(INPUT).iterdir(), total=len(os.listdir(INPUT))):
    if directory.name == ".DS_Store":
        continue
    !mkdir -p "{OUTPUT}/{directory.name}"
    file_paths = [f for f in directory.iterdir() if f.name != ".DS_Store"]
    with Pool(NUM_WORKERS // 2) as p:
        p.map(audio2vec, file_paths)
        #p.map(audio2pict, file_paths)
        #p.map(audio2quantized, file_paths)

### check ignore files

In [ ]:
for directory in tqdm.tqdm_notebook(Path(OUTPUT).iterdir(), total=len(os.listdir(OUTPUT))):
    if directory.name == ".DS_Store":
        continue
    file_paths = [f for f in directory.iterdir() if f.name != ".DS_Store"]
    for path in file_paths:
        size = os.path.getsize(path)
        if size < 1:
            print(path)

In [ ]:
paths = [
    f"{INPUT}/comrav/XC246425.wav",
    f"{INPUT}/prawar/XC479026.wav",
    f"{INPUT}/snobun/XC487557.wav",
    f"{INPUT}/snobun/XC487556.wav",
    f"{INPUT}/stejay/XC503349.wav"
]

In [ ]:
x, _ = sf.read(paths[0])
x_spex = audio_to_spec(x)

print(x_spex.shape)
cv2.imwrite(f"tmp.jpg", mono_to_color(x_spex))

### Audio Detection

In [ ]:
from matplotlib import pyplot as plt
import torch

In [ ]:
arr = np.load("../output/train_npz/aldfly/XC134874.wav.npz.npy")

In [ ]:
plt.plot(arr)

In [ ]:
arr.shape

In [ ]:
spect = librosa.feature.melspectrogram(arr.astype(float), sr=SAMPLE_RATE, fmin=20, fmax=16000, n_mels=128)

In [ ]:
plt.plot(spect.max(0))

In [ ]:
x = torch.Tensor(spect)
x = x.unsqueeze(0)
x.shape

In [ ]:
h = torch.nn.MaxPool1d(32)(x)
h.shape

In [ ]:
plt.plot(h[0].numpy().max(0))

In [ ]:
plt.plot(h[0].numpy().max(0) > 0.1*1e7)

### noise analysis

In [ ]:
from matplotlib import pyplot as plt
import IPython

In [ ]:
recs = defaultdict(list)
for directory in tqdm.tqdm_notebook(Path(INPUT).iterdir(), total=len(os.listdir(INPUT))):
    if directory.name == ".DS_Store":
        continue
    file_paths = [f for f in directory.iterdir() if f.name != ".DS_Store"]
    break

In [ ]:
path = file_paths[4]
path = "../input/train_resampled/ameavo/XC304534.wav"
print(path)
data, sr = librosa.load(path=path, sr=SAMPLE_RATE, mono=True)
plt.plot(data[:160000], ',', linestyle="None");plt.show()

x_spex = audio_to_spec(data)
pct = mono_to_color(x_spex)
plt.imshow(pct);plt.show()

IPython.display.Audio(data=data, rate=sr)

In [ ]:
N = len(data)
dt = 32000

F = np.fft.fft(data)
F_abs = np.abs(F)
F_abs_amp = F_abs / N * 2

fq = np.linspace(0, 1.0/dt, N)

plt.xlabel('freqency(Hz)', fontsize=14)
plt.ylabel('amplitude', fontsize=14)
plt.plot(fq, F_abs_amp)

In [ ]:
fc = 1e-5 # カットオフ（周波数）
F[(fq > fc)] = 0

#ac = 0.00002 # 振幅強度の閾値
#F[(F_abs_amp < ac)] = 0

F_abs = np.abs(F)
F_abs_amp = F_abs / N * 2

plt.xlabel('freqency(Hz)', fontsize=14)
plt.ylabel('amplitude', fontsize=14)
plt.plot(fq, F_abs_amp)

In [ ]:
F2_ifft = np.fft.ifft(F)
F2_ifft_real = F2_ifft.real * 2

IPython.display.Audio(data=F2_ifft_real, rate=sr)

### More Infomation

In [ ]:
import pandas as pd
train = pd.read_csv("../input/train.csv")
train.head(3)

In [ ]:
train["multi_label"] = train.apply(lambda x: [x["primary_label"]] + eval(x["secondary_labels"]) ,axis=1)

primary_label2ebird_code = {
    df["primary_label"].unique()[0]: ebird_code 
    for ebird_code, df in train[["ebird_code", "primary_label"]].groupby("ebird_code")
}

lst = []
for multi_label in train["multi_label"]:
    _lst = []
    for lab in multi_label:
        try:
            code = primary_label2ebird_code[lab]
        except KeyError:
            continue
        _lst.append(code)
    lst.append(_lst)
train["multi_ebird_code"] = lst

In [ ]:
def type2label(t):
    t = t.lower()
    d = [int("call" in t), int("song" in t)]
    return d

train["type_label"] = train["type"].map(type2label)

In [ ]:
train[["multi_ebird_code", "type_label"]].sample(10)